---
title: "Ames Housing Dashboard"
format: dashboard
page-navigation: true
---


::: {.column-page}

## Lasso 분석


In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
df = pd.read_excel('../find-my-home/ames_df.xlsx')

# 전처리###############################################################
# 수치형 변수 qual, cond 가중치줘서 새로운 열 추가
# 가중치 주기 위해 상관계수 분석
df[['SalePrice', 'OverallQual', 'OverallCond']].corr()  ## Qual이 상관계수 높게 나와 Qual가중치를 7로 줌

# Overall 점수 계산 (OverallQual 70%, OverallCond 30%)
df['Overall'] = df['OverallQual'] * 0.7 + df['OverallCond'] * 0.3
df


# 범주형 변수 qual, cond 가중치줘서 새로운 열 추가
# 점수화 기준 (543210 스케일)
qual_map_543210 = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1,
    'None': 0,
    'nan': 0,
    '0': 0
}

# 대상 변수
qual_vars = [
    "ExterQual", "ExterCond",
    "BsmtQual", "BsmtCond",
    "HeatingQC",
    "GarageQual", "GarageCond"
]



for col in qual_vars:
    df[col] = df[col].astype(str).replace(['nan', 'NaN', '0'], 'None')  # 예외처리 강화
    df[col + "_Score"] = df[col].map(qual_map_543210)

# 결과 일부 확인
df[[col + "_Score" for col in qual_vars]]

# 퀄리티 상관관계 확인
df[["SalePrice", "OverallQual", "OverallCond"]].corr()
df[["SalePrice", "ExterQual_Score", "ExterCond_Score"]].corr()
df[["SalePrice", "GarageQual_Score", "GarageCond_Score"]].corr()
df[["SalePrice", "BsmtQual_Score", "BsmtCond_Score"]].corr()

# 범주형 데이터 가중치 열 추가
df['Exter'] = df['ExterQual_Score'] * 0.9 + df['ExterCond_Score'] * 0.1
df['Garage'] = df['GarageQual_Score'] * 0.7 + df['GarageCond_Score'] * 0.3
df["Bsmt"] = df["BsmtQual_Score"] * 0.7 + df["BsmtCond_Score"] * 0.3
df.info()

cols_to_drop = [
    'OverallQual', 'OverallCond',
    'ExterQual', 'ExterCond',
    'BsmtQual', 'BsmtCond',
    'GarageQual', 'GarageCond',
    'ExterQual_Score', 'ExterCond_Score',
    'BsmtQual_Score', 'BsmtCond_Score',
    'GarageQual_Score', 'GarageCond_Score'
]

df = df.drop(columns=cols_to_drop)


# 예산 필터링
df = df[df['SalePrice'] >= 130000]
df = df[df['SalePrice'] <= 200000]

# x, y 분리! 
X = df.drop(columns='SalePrice')
y = (df['SalePrice'])

# X -> 수치형, 범주형 분리
num_columns = X.select_dtypes(include=['number']).columns
cat_columns = X.select_dtypes(include=['object']).columns

# 범주형은 원핫, 수치형은 스케일링 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
onehot = OneHotEncoder(handle_unknown='ignore', 
                       sparse_output=False)
X_train_cat = onehot.fit_transform(X[cat_columns])

std_scaler = StandardScaler()
X_train_num = std_scaler.fit_transform(X[num_columns])

X_train_all = np.concatenate([X_train_num, X_train_cat], axis = 1)


# LassoCV -> 가격에 많은 영향을 미치는 변수 찾기
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

alpha = np.linspace(0, 0.5, 1000)
lasso_cv = LassoCV(alphas=alpha,
                   cv=5,
                   max_iter=1000)
lasso_cv.fit(X_train_all, y)
lasso_cv.alpha_     # 아래 계산한 것들 평균내서 최적의 람다값 찾은 것
lasso_cv.mse_path_
lasso_cv_coef = lasso_cv.coef_

# 1. 원핫 범주형 변수 이름 뽑기
cat_feature_names = onehot.get_feature_names_out(cat_columns)

# 2. 전체 변수 이름 (수치형 + 범주형)
feature_names = np.concatenate([num_columns, cat_feature_names])

# 3. LassoCV에서 나온 계수와 변수이름 매칭
lasso_coef = lasso_cv.coef_

# 4. DataFrame으로 정리 + 절대값 기준 정렬
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': lasso_coef
})
coef_df['AbsCoefficient'] = coef_df['Coefficient'].abs()
coef_df = coef_df.sort_values('AbsCoefficient', ascending=False)

print(coef_df)

# 0인 값 제거
coef_df = coef_df[coef_df['Coefficient'] != 0]
coef_df = coef_df.sort_values('Coefficient', ascending=False)
coef_df.shape


# 예시: df_sorted는 Feature, Coefficient 등이 포함된 정리된 결과 데이터프레임
coef_df['Prefix'] = coef_df['Feature'].apply(lambda x: x.split('_')[0] if '_' in x else x)

# 그룹별로 묶기 (예: 평균/합계/갯수 등 집계도 가능)
grouped = coef_df.groupby('Prefix')

# 예시: 그룹별 Feature 개수 확인
print(grouped.size())

# 예시: 그룹별 Coefficient 총합 보기   
print(grouped['AbsCoefficient'].mean().sort_values(ascending=False))

In [ ]:
#| echo: false
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

# 데이터 로드 및 전처리
# (이전의 전체 코드가 여기에 삽입될 수 있음)

# 중요도 시각화
top_n = 20
coef_df = coef_df.sort_values('Coefficient', ascending=False).head(top_n)
plt.figure(figsize=(10, 6))
plt.barh(coef_df['Feature'], coef_df['Coefficient'])
plt.xlabel('Coefficient')
plt.title('Top 20 Important Features by Lasso Regression')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 품질 점수 산점도


In [ ]:
#| echo: false
import seaborn as sns
score_vars = ['Overall', 'Exter', 'Garage', 'Bsmt']
for var in score_vars:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(data=df, x=var, y='SalePrice')
    plt.title(f'{var} vs SalePrice')
    plt.tight_layout()
    plt.show()

## 상관계수 Heatmap


In [ ]:
#| echo: false
num_columns = df.select_dtypes(include=['number']).columns.drop('SalePrice')
top_corr_features = df[num_columns].corrwith(df['SalePrice']).abs().sort_values(ascending=False).head(10).index
plt.figure(figsize=(10, 8))
sns.heatmap(df[top_corr_features.to_list() + ['SalePrice']].corr(), annot=True, cmap='coolwarm')
plt.title("Top Correlated Numeric Features")
plt.tight_layout()
plt.show()

## 지도 시각화


In [ ]:
#| echo: false
import plotly.express as px
fig = px.scatter_mapbox(
    df,
    lat="Latitude",
    lon="Longitude",
    size="PriceNorm",
    color="SalePrice",
    color_continuous_scale="Viridis",
    size_max=50,
    zoom=11,
    hover_name="Neighborhood",
    hover_data={"SalePrice": True, "Exter": True, "Garage": True, "Bsmt": True, "Overall": True},
    title="🏠 Ames 주택 가격 지도 (정규화된 가격 기반)",
    mapbox_style="open-street-map"
)
fig.update_layout(mapbox_center={"lat": 42.03, "lon": -93.62}, mapbox_zoom=12, margin={"r":0,"t":40,"l":0,"b":0})
fig.show()

:::